# Qoffee Maker

In [ ]:
import os, time
from random import randint
from dotenv import load_dotenv
import numpy as np
load_dotenv();

In [ ]:
from ibm_quantum_widgets import CircuitComposer
from qiskit.circuit import QuantumCircuit
from qiskit import transpile, IBMQ, execute
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator
from qiskit import IBMQ
from qiskit.providers.ibmq.exceptions import IBMQAccountError
from qiskit.test.mock import FakeMontreal

## Authenticate to IoT Backend

In [1]:
%%html
<a class="btn btn-primary" target="_blank" href="/auth">Login</a>

In [2]:
%%html
<a class="btn btn-primary" onclick="window.activateCoffeeMachine()">Activate Coffee Machine</a>

## Setup Circuits

In [ ]:
try:
    IBMQ.enable_account(os.getenv("IBMQ_API_KEY"))
except IBMQAccountError as e:
    print("Got IBMAccountError", e, "probably due to re-execution")

provider = IBMQ.get_provider(hub="ibm-q-community", group="presentations")
provider.backends()

In [ ]:
backend_ideal = StatevectorSimulator()
backend_qasm = QasmSimulator()
backend_mock = FakeMontreal()
#backend_ibm = None 
backend_ibm = provider.get_backend("ibmq_jakarta")

## Circuits

In [ ]:
# empty
qc0 = QuantumCircuit(3, 3)

# cappucino
qc1 = QuantumCircuit(3, 3)
qc1.x(2)

# coffee
qc2 = QuantumCircuit(3, 3)
qc2.x(0)
qc2.h(1)
qc2.z(1)
qc2.h(1)

# espresso
qc3 = QuantumCircuit(3, 3)
qc3.x(0)
qc3.cx(0, 1)
qc3.cx(1, 0)
qc3.cx(0, 1)

# anything
qc4 = QuantumCircuit(3, 3)
qc4.h(0)
qc4.h(1)
qc4.h(2)

# without coffein
qc5 = QuantumCircuit(3, 3)
qc5.h(0)

# nothing strong
qc6 = QuantumCircuit(3, 3)
qc6.h(0)
qc6.x(2)

# tea or coffee
qc7 = QuantumCircuit(3, 3)
qc7.h(0)
qc7.cx(0, 1);


In [ ]:
# gather circuits with some meta information in one object
qcreg = [
    {
        "id": "empty",
        "name": "Start the Qoffee Maker",
        "circuit": qc0,
        "init_text": ""
    },
    {
        "id": "cappucino",
        "name": "Only Cappucino",
        "circuit": qc1,
        "init_text": ""
    },
    {
        "id": "coffee",
        "name": "Only Coffee",
        "circuit": qc2,
        "init_text": ""
    },
    {
        "id": "espresso",
        "name": "Only Espresso",
        "circuit": qc3,
        "init_text": ""
    },
    {
        "id": "anything",
        "name": "Just something to drink",
        "circuit": qc4,
        "init_text": ""
    },
    {
        "id": "withoutcoffein",
        "name": "Without caffein",
        "circuit": qc5,
        "init_text": ""
    },
    {
        "id": "nothingstrong",
        "name": "Nothing Strong",
        "circuit": qc6,
        "init_text": ""
    },
    {
        "id": "teaorcoffee",
        "name": "Tea or Coffee",
        "circuit": qc7,
        "init_text": ""
    }
]

In [ ]:
class CircuitExecutor:
    
    BIT_ORDER = ['000', '001', '010', '011', '100', '101', '110', '111']
    
    def __init__(self, circuit):
        """
        Helper Class to execute circuits on different backends.
        """
        self.circuit = circuit
        
    def _fill_with_zero(results):
        """
        Make sure that all keys defined in BIT_ORDER are present in the resulting dict.
        """
        for bit in CircuitExecutor.BIT_ORDER:
            if bit not in results:
                results[bit] = 0
        return results
    
    def _probabilities_backend(self, backend, num_shots=800, seed=None, result_options={}, transpile_before=False):
        """
        Calculate probabilities on a given backend.
        """
        # copy current circuit
        mqc = self.circuit.copy()
        # add measurement
        mqc.measure(range(3), range(3))
        # transpile
        mqc_transpiled = mqc if not transpile_before else transpile(mqc, backend)
        # run job
        job = backend.run(mqc_transpiled, shots=num_shots, seed=seed)
        # get results
        result = job.result(**result_options)
        counts = result.get_counts(mqc_transpiled)
        # divide by num_shots to get probabilities
        assert num_shots == np.sum([c for c in counts.values()]), (num_shots, counts)
        for b in counts:
            counts[b] = float(counts[b])/num_shots
        return CircuitExecutor._fill_with_zero(counts)
    
    
    def get_maximum_key(results):
        """
        Get the key which has the highest maximum
        """
        max_key = max(results, key=results.get)
        return max_key
    
    
    def probabilities_qasm(self, num_shots=800):
        """
        Calculate probabilities on QASM Simulator
        """
        return self._probabilities_backend(backend_qasm, num_shots=num_shots, seed=randint(0, 2500), transpile_before=True)
    
    
    def probabilities_mock(self, num_shots=800):
        """
        Calculate probabilities on Mock Device
        """
        return self._probabilities_backend(backend_mock, num_shots=num_shots, transpile_before=True)
    
    
    def probabilities_ibmq(self, num_shots=1, secs_timeout=30):
        """
        Calculate probabilities on IBM Q. If errors, fall back to mock device.
        """
        try:
            res = self._probabilities_backend(backend_ibm, num_shots=num_shots, result_options={"timeout": secs_timeout, "wait": 2}, transpile_before=True)
            return res
        except Exception as e:
            print("Got error for running on IBM Q", e)
            return self.probabilities_mock(num_shots=num_shots)
    
    
    def probabilities_analytical(self):
        """
        Calculate probabilities using the state vector (analytical solution)
        """
        mqc = self.circuit.copy()
        # run circuit on state vector
        result = execute(mqc, backend_ideal).result()
        # get state vector and calculate probabilities
        state_vector = result.get_statevector(mqc, decimals=5)
        probs = state_vector.probabilities() if hasattr(state_vector, "probabilities") else np.abs(state_vector ** 2)
        # map back to bit configurations
        results = {
            key: probs[i] for i, key in enumerate(CircuitExecutor.BIT_ORDER)
        }
        return CircuitExecutor._fill_with_zero(results)

## Build App

In [ ]:
from ipywidgets import Widget, Button, HTML, VBox, widgets, link, HBox
import appwidgets
from traitlets import Unicode, Dict, Int, Any, HasTraits, Bool
from appwidgets import JsPyWidget
import json

### Build Data Model

In [ ]:
# gather drink metadata
DRINKS = {
    "000": {
        "id": "tea",
        "name": "Tea",
        "key": "NotImplemented",
        "options": {}
    },
    "001": {
        "id": "hotchocolate",
        "name": "Hot Chocolate",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.MilkFroth",
        "options": {}
    },
    "010": {
        "id": "espresso",
        "name": "Espresso",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.Espresso",
        "options": {
            "ConsumerProducts.CoffeeMaker.Option.FillQuantity": 50
        }
    },
    "011": {
        "id": "coffee",
        "name": "Coffee",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.Coffee",
        "options": {}
    },
    "100": {
        "id": "cappucino",
        "name": "Cappucino",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.Cappuccino",
        "options": {}
    },
    "101": {
        "id": "lattemacchiato",
        "name": "Latte Macchiato",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.LatteMacchiato",
        "options": {}
    },
    "110": {
        "id": "wienermelange",
        "name": "Viennese Melange",
        "key": "ConsumerProducts.CoffeeMaker.Program.CoffeeWorld.WienerMelange",
        "options": {}
    },
    "111": {
        "id": "americano",
        "name": "Americano",
        "key": "ConsumerProducts.CoffeeMaker.Program.CoffeeWorld.Americano",
        "options": {}
    }
}

In [ ]:
# build a widget which represents the global app state
data = Widget()
data.add_traits(
    # name of current circuit with which the composer was initialized
    composer_init_circuit = Dict({
        "id": "empty",
        "name": "",
        "init_text": ""
    }).tag(sync=True),
    # current ideal probabilities
    histogram_anal = Dict({
        key: 0 for key in DRINKS
    }).tag(sync=True),
    # current probabilities from ideal simulator
    histogram_qasm = Dict({
        key: 0 for key in DRINKS
    }).tag(sync=True),
    # current probabilities from mock device
    histogram_mock = Dict({
        key: 0 for key in DRINKS
    }).tag(sync=True),
    # id of drink from single shot measurement
    single_shot_result = Dict({
        "name": "",
        "id": "",
        "bit": ""
    }).tag(sync=True),
    # status of single shot measurements (0 = init, 1 = loading, 2 = done)
    single_shot_status = Int().tag(sync=True),
    # device on which to perform the single shot
    single_shot_device = Unicode("simulator").tag(sync=True),
    # composer qasm
    qasm = Unicode("").tag(sync=True),
    # check if real device is enabled
    use_real_device = Bool(backend_ibm is not None).tag(sync=True),
    # check if mock device results are shown
    show_mock_results = Bool(False).tag(sync=True),
    # check if qasm results are shown
    show_qasm_results = Bool(False).tag(sync=True),
    
    #### VIEW RELATED
    view = Unicode("").tag(sync=True),
    heading = Unicode("").tag(sync=True)
)

### Setup the composer

In [ ]:
composer = CircuitComposer(circuit=qcreg[0]['circuit'])

In [ ]:
# on changes on the composer, execute the circuit to get probabilities
def composer_update_handler(change_event):
    # there are multiple change events, only use on
    if change_event['name'] != "circuit":
        return
    executor = CircuitExecutor(composer.circuit)
    # set to 0 because calc requires some time and one sees wrong results for a second or so
    data.histogram_mock = {key: 0 for key in CircuitExecutor.BIT_ORDER}
    # get and save ideal result
    result_analytical = executor.probabilities_analytical()
    data.histogram_anal = result_analytical
    # get and save measurements
    result_qasm = executor.probabilities_qasm()
    data.histogram_qasm = result_qasm
    # get and save qasm code
    qasm = composer.circuit.qasm()
    data.qasm = qasm
    
    # most time consuming :/
    if data.show_mock_results:
        result_mock = executor.probabilities_mock()
        data.histogram_mock = result_mock
    
composer.observe(composer_update_handler)

# execute the function above also on changes on data.show_mock_results and data.show_qasm_results
data.observe(lambda x: composer_update_handler({"name": "circuit"}), names=["show_mock_results", "show_qasm_results"])

### Build app logic

In [ ]:
# perform single shot measurement
def get_single_shot_result():
    data.single_shot_status = 1
    executor = CircuitExecutor(composer.circuit)
    counts = None
    if data.single_shot_device == "ibmq":
        counts = executor.probabilities_ibmq(num_shots=1, secs_timeout=30)
    elif data.single_shot_device == "mock":
        counts = executor.probabilities_mock(num_shots=1)
    else:
        counts = executor.probabilities_qasm(num_shots=1)
        
    bits = CircuitExecutor.get_maximum_key(counts)
    data.single_shot_result = {
        "bit": bits,
        "id": DRINKS[bits]["id"],
        "name": DRINKS[bits]["name"]
    }
    time.sleep(1.5)
    data.single_shot_status = 2
    return bits

In [ ]:
# request a drink from the backend
def request_drink():
    # get result from single shot measurement
    bit = data.single_shot_result['bit']
    drink_key = DRINKS[bit]['key']
    drink_options = DRINKS[bit]['options']
    
    def response_handler(res):
        if 'error' in res:
            # app.view = 'error'
            data.single_shot_status = 4
        else:
            # app.view = 'success'
            data.single_shot_status = 3
    
    jspy.execute_js(
        "window.requestDrink("+json.dumps(drink_key)+", "+json.dumps(drink_options)+")",
        response_handler
    )

In [ ]:
# show the QR Code to open IBM Q with the current circuit
def open_circuit_on_ibmq():
    qasm = data.qasm
    jspy.execute_js(
        "window.openQRCodeIBMQ("+json.dumps(qasm)+")",
    )

In [ ]:
# toggle a boolean attribute on data
def toggle_data_attr(name):
    setattr(data, name, not getattr(data, name))
    
    
# toggle device to use for single shots
def set_next_single_shot_device():
    if data.single_shot_device == 'simulator':
        data.single_shot_device = 'mock'
    elif data.single_shot_device == 'mock':
        if data.use_real_device:
            data.single_shot_device = 'ibmq'
        else:
            data.single_shot_device = 'simulator'
    elif data.single_shot_device == 'ibmq':
        data.single_shot_device = 'simulator'
    else:
        data.single_shot_device = 'simulator'

In [ ]:
## functions to load specific views

# load welcome view
def load_welcome():
    data.view = 'welcome'
    data.heading = "Qoffee Maker powered by <b>IBM Quantum</b>"
    

# load starting point (set circuit in composer, reset some attributes, ...)
def load_starting_point(circuit_id):
    circuit_data = next(filter(lambda x: x['id'] == circuit_id, qcreg))
    data.composer_init_circuit = {
        "id": circuit_data["id"],
        "name": circuit_data["name"],
        "init_text": circuit_data["init_text"]
    }
    data.single_shot_device = 'simulator'
    data.single_shot_status = 0
    composer.circuit = circuit_data['circuit']
    data.show_mock_results = False
    data.show_qasm_results = False
    # data.heading = "Composer: "+circuit_data["name"]
    data.view = 'composer'

### Build views

In [ ]:
app = appwidgets.AppBox(init_view='welcome')
link((app, "view"), (data, "view"))
jspy = JsPyWidget()

#### Header

In [ ]:
header = appwidgets.ReactiveHtmlWidget("""
    <div id="header">
        <img src="css/QoffeeMug.png"/>
        <h1>${heading}</h1>
        <div id="navigation-container">
            <button id="navigation-button-q" data-device="${single_shot_device}" data-rh-exec='set_next_single_shot_device()'>Q</button>
            <button id="navigation-button-back" data-rh-if="${view} != 'welcome'" class="navigation-button" data-rh-exec="load_welcome()">&#5130;</button>
            <button id="navigation-button-help" data-rh-exec-js='window.openQRCode("http://qoffee-maker.org", "http://qoffee-maker.org")'>?</button>
        </div>
    </div>
""", data_model=data)
header.load_css("./css/header.css")
data.heading = "Qoffee Maker powered by <b>IBM Quantum</b>"

#### Welcome View

In [ ]:
# helper function to get html code
def get_circuit_selection_html(circuit_id, width=12):
    circuit_data = next(filter(lambda x: x['id'] == circuit_id, qcreg))
    return """
    <div class="col-md-"""+str(width)+"""">
        <div class="card" data-rh-exec="load_starting_point('"""+circuit_id+"""')">
          <div class="card-body">
            <h5 class="card-title">"""+circuit_data['name']+"""</h5>
            <p class="card-text">"""+circuit_data['init_text']+"""</p>
            <span class="arrow">→</span>
          </div>
        </div>
    </div>
    """

In [ ]:
view_welcome_content = appwidgets.ReactiveHtmlWidget("""
    <div id="view-welcome" class="app-view-content">
        <h3>Welcome to the Qoffee Maker</h3>
        <br/>
        <div class="row mt-4">
            <div class="col-md-12">
                <div class="row align-items-stretch card-primary">"""+
                    get_circuit_selection_html('empty', 3)
                +"""
                </div>
            </div>
        </div>
        <br/>
        <br/>
        <br/>
        <div class="row mt-4">
            <div class="col-md-12">
                <h4>Start with the examples from qoffee-maker.org</h4>
                <div class="row align-items-stretch">"""+
                    get_circuit_selection_html('anything', 3) +
                    get_circuit_selection_html('cappucino', 3) +
                    get_circuit_selection_html('withoutcoffein', 3) + 
                    get_circuit_selection_html('nothingstrong', 3) + 
                    get_circuit_selection_html('teaorcoffee', 3)
                +"""
                </div>
            </div>
        </div>
    </div>
""")

view_welcome_content.load_css("./css/view_welcome.css")

In [ ]:
view_welcome = VBox([header, view_welcome_content])

#### Composer View

In [ ]:
# Introtext
#view_composer_intro = appwidgets.ReactiveHtmlWidget("""
#    <p>{{${composer_init_circuit}['init_text']}}</p>
#""", data_model=data)
view_composer_intro = appwidgets.ReactiveHtmlWidget("""
    <p>Drag & Drop Quantum Gates onto the Qubits to build your circuit. <span data-rh-exec="load_welcome()">Go back</span>.</p>
""")
view_composer_intro.load_css("./css/view_composer_intro.css")

# Histogram
hist_html = ""
for key, value in DRINKS.items():
    hist_html += """
        <div class="histogram-entry">
            <div class="histogram-text-container-pre">
                <p><span class="histogram-text histogram-text-ideal histogram-text-"""+key+""""">{{int(${histogram_anal}['"""+key+"""']*1000)/10}}%</span></p>
            </div>
            <div class="histogram-bar-container">
                <div class="histogram-bar-outer">
                    <div data-rh-if="${show_qasm_results}" style="height: {{${histogram_qasm}['"""+key+"""']*100}}%" class="histogram-bar histogram-bar-qasm histogram-bar-"""+key+""""></div>
                    <div style="height: {{${histogram_anal}['"""+key+"""']*100}}%" class="histogram-bar histogram-bar-anal histogram-bar-"""+key+""""></div>
                    <div data-rh-if="${show_mock_results}" style="height: {{${histogram_mock}['"""+key+"""']*100}}%" class="histogram-bar histogram-bar-mock histogram-bar-"""+key+""""></div>
                </div>
            </div>
            <div class="histogram-text-container-post">
                <p>"""+key+"""<br/><br/>"""+value["name"]+"""</p>
            </div>
        </div>
    """
    
view_composer_button_bar = appwidgets.ReactiveHtmlWidget("""
    <div id="view-composer-hist-header">
        <h3>Measurement Probabilities</h3>
    </div>
    <div id="view-composer-button-bar">
        <button id="view-composer-show-theo-btn" data-active="True" class="btn btn-default">Theoretical</button>
        <button id="view-composer-show-qasm-btn" data-active="${show_qasm_results}" class="btn btn-default" data-rh-exec="toggle_data_attr('show_qasm_results')">Simulator (error free)</button>
        <button id="view-composer-show-mock-btn" data-active="${show_mock_results}" class="btn btn-default mr-4" data-rh-exec="toggle_data_attr('show_mock_results')">Simulator (real quantum device)</button>
        <button class="btn btn-default mr-4" data-rh-exec="open_circuit_on_ibmq()">Export to IBM Quantum Composer</button>
    </div>
    
""", data_model=data)
view_composer_button_bar.load_css("./css/view_composer_button_bar.css")

view_composer_hist = appwidgets.ReactiveHtmlWidget("""
<div id="view-composer-hist-container">
    <div id="view-composer-hist">"""+hist_html+"""</div>
</div>
""", data_model=data)
view_composer_hist.load_css("./css/view_composer_hist.css")

# Single Shot
view_composer_singleshot = appwidgets.ReactiveHtmlWidget("""
    <div id="view-composer-singleshot-container">
        <div id="view-composer-singleshot">
            <button class="btn btn-primary btn-block" data-rh-exec="get_single_shot_result()">Determine your beverage</button>
            <br/>
            <div data-rh-if='${single_shot_status}==1' class="width-100">
                <p style="width: 100%; text-align: center;">Loading...
            </div>
            <div id="view-composer-singleshot-result"data-rh-if='${single_shot_status}==2' class="view-composer-order-text width-100">
                <p>☕️<br>{{${single_shot_result}["name"]}}</p>
                <button class="btn btn-primary btn-block" data-rh-exec="request_drink()">Order Drink →</button>
            </div>
            <div data-rh-if='${single_shot_status}==3' class="view-composer-order-text width-100">
                <p>👍<br>Thank you and enjoy your drink</p>
                <button class="btn btn-primary btn-block" data-rh-exec="load_welcome()">Back</button>
            </div>
            <div data-rh-if='${single_shot_status}==4' class="view-composer-order-text width-100">
                <p>😱<br>Oh no, something went wrong.</p>
                <button class="btn btn-primary btn-block" data-rh-exec="load_welcome()">Back</button>
            </div>
            <br/>
            
        </div>
    </div>
""", data_model=data)
view_composer_singleshot.load_css("./css/view_composer_singleshot.css")

In [ ]:
# assemble composer view
view_composer = VBox([
    header,
    VBox([
        view_composer_intro,
        composer,
        HBox([
            VBox([
                view_composer_button_bar,
                view_composer_hist
            ]).add_class("width-80"),
            view_composer_singleshot.add_class("width-20")
        ])
    ]).add_class("app-view-content")  
])

#### Error View

In [ ]:
view3_content = appwidgets.ReactiveHtmlWidget("""
    <div class="app-view-content">
        <h1>Oh no, something went wrong 😱</h1>
    </div>
""")

view_fail = VBox([
    header,
    view3_content
])

#### Success View

In [ ]:
view4_content = appwidgets.ReactiveHtmlWidget("""
    <div class="app-view-content">
        <h1>Thank you and enjoy your drink ☕️</h1>
    </div>
""")

view_success = VBox([
    header,
    view4_content
])

### Assemble View

In [ ]:
app.add_widget('welcome', view_welcome)
app.add_widget('composer', view_composer)
app.add_widget('error', view_fail)
app.add_widget('success', view_success)

# Show App

In [ ]:
### APP
app